In [1]:
# essential libraries
import os
from datetime import datetime as dt

# storing and anaysis
import numpy as np
import pandas as pd

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# visualization
# matplot and sns 나중에 안쓰면 지우기
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image

# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

#한글깨짐 방지(0420)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

#pandas와 같이 plotly에서도 간단한 시각화 지원
import cufflinks as cf
cf.go_offline(connected=True)

# for offline ploting
# ===================
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
path = '../covid'
os.chdir('data')
file_name_list = os.listdir()

In [3]:
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']
layout_setting = {'xaxis_title':'Date','yaxis_title':'Number','font':dict(size=18,color='#60606e',family='Franklin Gothic' )}

In [4]:
timeProvince = pd.read_csv('TimeProvince.csv',)

In [5]:
timeProvince

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0
...,...,...,...,...,...,...
1559,2020-04-20,0,Jeollabuk-do,17,9,0
1560,2020-04-20,0,Jeollanam-do,15,8,0
1561,2020-04-20,0,Gyeongsangbuk-do,1361,1064,51
1562,2020-04-20,0,Gyeongsangnam-do,116,87,0


In [6]:
confirmed = pd.pivot_table(timeProvince,index='date',columns='province',values='confirmed')
released = pd.pivot_table(timeProvince,index='date',columns='province',values='released')
deceased = pd.pivot_table(timeProvince,index='date',columns='province',values='deceased')

In [7]:
isolation = confirmed-released-deceased

In [8]:
confirmed_diff = confirmed.diff().fillna(0)
confirmed_diff = pd.DataFrame(np.where(confirmed_diff<0,-1*confirmed_diff,confirmed_diff),index=confirmed_diff.index, columns=confirmed_diff.columns)

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

confirmed_s = pd.DataFrame(scaler.fit_transform(confirmed), index=confirmed.index, columns=confirmed.columns)
released_s = pd.DataFrame(scaler.fit_transform(released), index=released.index, columns=released.columns)
deceased_s = pd.DataFrame(scaler.fit_transform(deceased), index=deceased.index, columns=deceased.columns)
confirmed_diff_s = pd.DataFrame(scaler.fit_transform(confirmed_diff),index=confirmed_diff.index, columns=confirmed_diff.columns)

In [10]:
provinces = ['Busan','Daegu','Gyeongsangnam-do','Gyeongsangbuk-do','Chungcheongnam-do','Chungcheongbuk-do','Gangwon-do','Jeollanam-do','Jeollabuk-do','Gwangju','Daejeon','Ulsan',
            'Jeju-do','Sejong','Incheon','Gyeonggi-do','Seoul']

fig = make_subplots(rows=6, cols=3, horizontal_spacing=0.03,
                    vertical_spacing= 0.05,
                   subplot_titles=([f'<b>{prov}</b>' for prov in provinces]))

for i, province in enumerate(provinces):
    row, col, legend = i//3 + 1, i%3 + 1, False
    if i == len(provinces)-1 :
        legend = True
    fig.add_trace(go.Scatter(x=confirmed_s.index, y=confirmed_s[province],
                     mode='lines', name="누적 확진자", 
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[3]), showlegend=legend), row=row, col=col)
    
    fig.add_trace(go.Bar(x=confirmed_diff_s.index, y=confirmed_diff_s[province],
                     name="신규 확진자",
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[9]), showlegend=legend), row=row, col=col)

fig.update_layout(title='<b>지역별 코로나 확진자 추이</b>',
                  height = 2000, font = layout_setting['font'],
                  legend=dict(x=0.7, y=0.05, traceorder="normal",
                             font=dict(family="sans-serif", size=18)))
for i in fig['layout']['annotations']:
    i['font'] = dict(size=20)

In [11]:
fig.show()

```
지역별 확진자 추이를 보았을 때, 코로나 확진자 수가 초반에 급증한 지역과 그러지 않는 지역으로, 총 두 가지 유형을 볼 수 있습니다.
서울, 경기, 인천외 지역에서는 꾸준히 확진자가 나오는 반면, 그 외에 지역에서는 초반에 확진자가 급증한 후 확진자 수가 대폭 감소하는 것울 볼 수 있습니다.
```

In [12]:
fig = make_subplots(rows=1, cols=4,
                   subplot_titles=([f'<b>{prov}</b>' for prov in provinces]))
provinces = ['Daegu','Gyeongsangbuk-do','Seoul','Gyeonggi-do']
for i, province in enumerate(provinces):
    row, col, legend = i//4 + 1, i%4 + 1, False
    if i == len(provinces)-1 :
        legend = True
    fig.add_trace(go.Scatter(x=confirmed_s.index, y=confirmed_s[province],
                     mode='lines', name="누적 확진자", 
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[3]), showlegend=legend), row=row, col=col)
    
    fig.add_trace(go.Bar(x=confirmed_diff_s.index, y=confirmed_diff_s[province],
                     name="신규 확진자",
                     marker=dict(color = DEFAULT_PLOTLY_COLORS[9]), showlegend=legend), row=row, col=col)

fig.update_layout(title='<b>대구, 경북, 서울, 경기 확진자 추이</b>',
                  font = layout_setting['font'],
                  legend=dict(x=0.7, y=0.05, traceorder="normal",
                             font=dict(family="sans-serif", size=18)))

```
확지자가 가장 많은 곳은 서울, 경기, 대구, 경북에서 이러한 현상을 뚜렷하게 볼 수 있습니다. 
대구와 경북이 초반에 확진자가 많았던 이유는 코로나 확진자의 대부분은 신천지 관련으로 집단으로 발생하였기 때문입니다.
최근 2주간 감염 경로별 신규 확진자 발생 현황을 보았을때, 해외 유입이 57%로 가장 높았고 감염경로가 해외유입인 확진자의 대부분이 서울과 경기입니다. 그러므로 서울과 경기는 다른 지역과 다르게 꾸준히 확진자가 나오고 있는 이유를 해외 유입 인구로 볼 수 있습니다.
```

In [13]:
fig.show()

In [14]:
PatiendtInfo = pd.read_csv('PatientInfo.csv')

In [15]:
oversea = PatiendtInfo[PatiendtInfo.infection_case=='overseas inflow']
oversea_province = pd.pivot_table(oversea, index='confirmed_date',columns='province',values='patient_id',aggfunc='count').fillna(0).cumsum()

fig = go.Figure() # Figure 객체 생성
for name in oversea_province.columns:
    fig.add_trace(go.Scatter(x=oversea_province.index, y=oversea_province[name], mode='lines', name=name))
fig.update_layout(title="<b>지역별 해외 유입 확진자 추이<b>", **layout_setting)

In [16]:
fig.show()

In [17]:
infection_case = pd.pivot_table(PatiendtInfo,index='confirmed_date',columns='infection_case',values='patient_id',aggfunc='count').fillna(0).cumsum()
infection_case = pd.DataFrame(scaler.fit_transform(infection_case), index=infection_case.index, columns=infection_case.columns)

fig = go.Figure()
for name in infection_case.columns:
    fig.add_trace(go.Scatter(x=infection_case.index, y=infection_case[name], mode='lines', name=name))
fig.update_layout(title="<b>감염 경로 추이<b>", **layout_setting)

In [18]:
fig.show()